In [1]:
import os
import pandas as pd
import pickle
import ast
import sys
import re

In [2]:
OUT_DIR="/media/SSD/paper_data/"

In [3]:
def get_loss_acc(input_file):

    with open(input_file, 'rb') as f:
        dest_object_name = pickle.load(f)

    train_loss = dest_object_name[dest_object_name.find("loss"):dest_object_name.find("]")+1]
    train_accuracy = dest_object_name[dest_object_name.find("accuracy"):dest_object_name.find("val_loss")-3]
    validation_loss = dest_object_name[dest_object_name.find("val_loss"):dest_object_name.find("val_accuracy")-3]
    validation_accuracy = dest_object_name[dest_object_name.find("val_accuracy"):dest_object_name.find("lr")-3]

    train_loss_list = ast.literal_eval(train_loss[train_loss.find("["):train_loss.find("]")+1])
    train_accuracy_list = ast.literal_eval(train_accuracy[train_accuracy.find("["):train_accuracy.find("]")+1])
    validation_loss_list = ast.literal_eval(validation_loss[validation_loss.find("["):validation_loss.find("]")+1])
    validation_accuracy_list = ast.literal_eval(validation_accuracy[validation_accuracy.find("["):validation_accuracy.find("]")+1])
    
    return train_accuracy_list, validation_accuracy_list

In [129]:
sensor_tag="all"
seeds = [0]
preprocessings = ["none", "CWT-H", "CWT-V"]
models = ["LSTM", "CNN", "CNN_small", "LSTM+CNN", "CNN+LSTM", "MLP", "resnet"]
datasets = ["IN/nops", "OUT1/nops", "OUT1/random", "OUT2/nops"]
i = 10000
topks = [2, 3, 4, 5, 6]
topk_empty = []

columns = ['dataset', 'model', 'preprocessing', 'seed', 'kfold', 'accuracy']
for topk in topks:
    columns.append("top-"+str(topk)+"-acc")
    topk_empty.append("-")
accuracies = pd.DataFrame(columns=columns)

empty_list = []
for acc_type in ['val', 'train']:
    for epoch in range(100):
        columns.append(acc_type+"_e"+str(epoch))
        empty_list.append('-')
accuracies_evolution = pd.DataFrame(columns=columns)

jobs_to_repeat = []

results_idx = 0

for seed in seeds:
    for preprocessing in preprocessings:
        for model in models:
            for dataset in datasets:
                
                model_small=model.replace("_", "-").replace("+", "-").replace("/", "-").lower()
                dataset_small=dataset.replace("_", "-").replace("+", "-").replace("/", "-").lower()
                preprocessing_small=preprocessing.replace("_", "-").replace("+", "-").replace("/", "-").lower()

                acc_file=OUT_DIR+"/sakura/Exp-"+dataset+"/results/"+model+"_p"+preprocessing+"_"+str(i)+"_s"+str(seed)+"-sensors-"+sensor_tag+"_10fold/out/kfold_results.txt"
                
                # Check if job was completed
                if(os.path.exists(acc_file)!=True):
                    print("Results for "+"Exp-"+dataset+"/results/"+model+"_p"+preprocessing+"_"+str(i)+"_s"+str(seed)+"-sensors-"+sensor_tag+"_10fold"+" not yet ready!")
                    jobs_to_repeat.append("prs-"+dataset_small+"-"+model_small+"-p"+preprocessing_small+"-"+str(i)+"-s"+str(seed)+"-sens-"+sensor_tag)
                    for kfold_tmp in range(10):
                        accuracies.loc[results_idx] = ["Exp-"+dataset_small, model, preprocessing, 0, kfold_tmp, "-"] + topk_empty
                        accuracies_evolution.loc[results_idx] = ["Exp-"+dataset_small, model, preprocessing, 0, kfold_tmp, "-"] + topk_empty + empty_list
                        results_idx = results_idx+1
                    accuracies.loc[results_idx] = ["Exp-"+dataset_small, model, preprocessing, 0, "avg", "-"] + topk_empty
                    accuracies_evolution.loc[results_idx] = ["Exp-"+dataset_small, model, preprocessing, 0, "avg", "-"] + topk_empty + empty_list
                    results_idx = results_idx+1
                    continue
                
                # Check if data was stored safetly
                num_lines = sum(1 for line in open(acc_file))
                if(num_lines <= 11):
                    print("Missing data for "+"Exp-"+dataset+"/results/"+model+"_p"+preprocessing+"_"+str(i)+"_s"+str(seed)+"-sensors-"+sensor_tag+"_10fold")
                    jobs_to_repeat.append("prs-"+dataset_small+"-"+model_small+"-p"+preprocessing_small+"-"+str(i)+"-s"+str(seed)+"-sens-"+sensor_tag)
                    for kfold_tmp in range(10):
                        accuracies.loc[results_idx] = ["Exp-"+dataset_small, model, preprocessing, 0, kfold_tmp, "-"] + topk_empty
                        accuracies_evolution.loc[results_idx] = ["Exp-"+dataset_small, model, preprocessing, 0, kfold_tmp, "-"] + topk_empty + empty_list
                        results_idx = results_idx+1
                    accuracies.loc[results_idx] = ["Exp-"+dataset_small, model, preprocessing, 0, "avg", "-"] + topk_empty
                    accuracies_evolution.loc[results_idx] = ["Exp-"+dataset_small, model, preprocessing, 0, "avg", "-"] + topk_empty + empty_list
                    results_idx = results_idx+1
                    continue
                #print(out_path)
                
                file1 = open(acc_file, 'r')
                Lines = file1.readlines()

                for count, line in enumerate(Lines):
                    #print("Line{}: {}".format(count, line.strip()))
                    if(count>=1 and count <=10):
                        acc_evol=OUT_DIR+"/sakura/Exp-"+dataset+"/results/"+model+"_p"+preprocessing+"_"+str(i)+"_s"+str(seed)+"-sensors-"+sensor_tag+"_10fold/out/log/LSTM+CNN_kf"+str(count-1)+".txt"
                        train_acc, test_acc = get_loss_acc(acc_evol)
                        kf_accuracy = re.findall("\d+\.\d+", line.strip())[1]
                        #row = pd.DataFrame(['Exp-IN', 'LSTM+CNN', 'none', 0, count, kf_accuracy])
                        #accuracies = pd.concat([accuracies, row], axis=0, ignore_index=True)
                        accuracies.loc[results_idx] = ["Exp-"+dataset_small, model, preprocessing, 0, count-1, kf_accuracy] + topk_empty
                        accuracies_evolution.loc[results_idx] = ["Exp-"+dataset_small, model, preprocessing, 0, count-1, kf_accuracy] + topk_empty + train_acc + test_acc
                        results_idx = results_idx+1
                    if(count==12):
                        kf_accuracy = re.findall("\d+\.\d+", line.strip())[0]
                        accuracies.loc[results_idx] = ["Exp-"+dataset_small, model, preprocessing, 0, "avg", kf_accuracy] + topk_empty
                        accuracies_evolution.loc[results_idx] = ["Exp-"+dataset_small, model, preprocessing, 0, "avg", kf_accuracy] + topk_empty + empty_list
                        results_idx = results_idx+1
                
                file1.close()
                
                # Collect top-K accuracies
                if(preprocessing == 'none'):
                    for topk in topks:
                        topk_file=OUT_DIR+"/sakura/Exp-"+dataset+"/results/"+model+"_p"+preprocessing+"_"+str(i)+"_s"+str(seed)+"-sensors-"+sensor_tag+"_10fold/out/top-"+str(topk)+"_kfold_results.txt"
                        if(os.path.exists(topk_file)!=True):
                            print("Top-"+str(topk)+" results for "+"Exp-"+dataset+"/results/"+model+"_p"+preprocessing+"_"+str(i)+"_s"+str(seed)+"-sensors-"+sensor_tag+"_10fold"+" not yet ready!")
                        else:
                            file1 = open(topk_file, 'r')
                            Lines = file1.readlines()

                            for count, line in enumerate(Lines):
                                #print("Line{}: {}".format(count, line.strip()))
                                if(count>=1 and count <=10):
                                    kf_accuracy = re.findall("\d+\.\d+", line.strip())[0]
                                    target_idx = accuracies.loc[  (accuracies.kfold==(count-1))
                                                                & (accuracies.dataset=="Exp-"+dataset_small)
                                                                & (accuracies.model==model)
                                                                & (accuracies.preprocessing==preprocessing)
                                                                & (accuracies.seed==seed)].index
                                    accuracies.loc[target_idx, 'top-'+str(topk)+'-acc'] = kf_accuracy
                                    accuracies_evolution.loc[target_idx, 'top-'+str(topk)+'-acc'] = kf_accuracy
                                if(count==11):
                                    kf_accuracy = re.findall("\d+\.\d+", line.strip())[0]
                                    target_idx = accuracies.loc[  (accuracies.kfold=='avg')
                                                                & (accuracies.dataset=="Exp-"+dataset_small)
                                                                & (accuracies.model==model)
                                                                & (accuracies.preprocessing==preprocessing)
                                                                & (accuracies.seed==seed)].index
                                    accuracies.loc[target_idx, 'top-'+str(topk)+'-acc'] = kf_accuracy
                                    accuracies_evolution.loc[target_idx, 'top-'+str(topk)+'-acc'] = kf_accuracy

In [149]:
# Save output files
accuracies.to_csv(OUT_DIR+"/sakura/accuracies/accuracies_all.csv", index=False)
accuracies.loc[accuracies.preprocessing=='none'].reset_index(drop=True).to_csv(OUT_DIR+"/sakura/accuracies/accuracies_kfold_topk.csv", index=False)
accuracies.loc[(accuracies.preprocessing=='none') & (accuracies.kfold=='avg')].reset_index(drop=True).to_csv(OUT_DIR+"/sakura/accuracies/accuracies_kfold_topk_averaged.csv", index=False)
accuracies.iloc[:, 0:6].to_csv(OUT_DIR+"/sakura/accuracies/accuracies_kfold.csv", index=False)
accuracies.iloc[:, 0:6].loc[accuracies.kfold=='avg'].reset_index(drop=True).to_csv(OUT_DIR+"/sakura/accuracies/accuracies_averaged.csv", index=False)
accuracies_evolution.to_csv(OUT_DIR+"/sakura/accuracies/accuracies_evolution_all.csv", index=False)

In [188]:
accuracies.loc[idx]

,dataset,model,preprocessing,seed,kfold,accuracy,top-2-acc,top-3-acc,top-4-acc,top-5-acc,top-6-acc
538,Exp-in-nops,MLP,CWT-H,0,avg,82.2475677728653,-,-,-,-,-
549,Exp-out1-nops,MLP,CWT-H,0,avg,65.47189235687256,-,-,-,-,-
560,Exp-out1-random,MLP,CWT-H,0,avg,54.63243246078491,-,-,-,-,-
571,Exp-out2-nops,MLP,CWT-H,0,avg,45.60513526201248,-,-,-,-,-


In [204]:
accuracies.loc[(accuracies.preprocessing=='none') & (accuracies.model == model)& (accuracies.dataset == "Exp-out1-random")]

,dataset,model,preprocessing,seed,kfold,accuracy,top-2-acc,top-3-acc,top-4-acc,top-5-acc,top-6-acc
286,Exp-out1-random,resnet,none,0,0,59.594595432281494,77.94324159622192,87.35405206680298,92.92972683906555,96.10000252723694,97.78918623924255
287,Exp-out1-random,resnet,none,0,1,59.85405445098877,77.98107862472534,87.22972869873047,92.77837872505188,96.11892104148865,97.88378477096558
288,Exp-out1-random,resnet,none,0,2,60.0162148475647,78.4702718257904,87.62162327766418,93.13783645629883,96.32973074913025,97.91892170906067
289,Exp-out1-random,resnet,none,0,3,59.791892766952515,78.20000052452087,87.61891722679138,92.85135269165039,95.9864854812622,97.73243069648743
290,Exp-out1-random,resnet,none,0,4,59.770268201828,78.27026844024658,87.38108277320862,92.97026991844177,96.18918895721436,97.82162308692932
291,Exp-out1-random,resnet,none,0,5,59.589189291000366,77.90811061859131,87.2891902923584,93.01891922950745,96.20810747146606,97.77837991714478
292,Exp-out1-random,resnet,none,0,6,60.18378138542175,77.93243527412415,87.2891902923584,92.91892051696777,96.11351490020752,97.81081080436707
293,Exp-out1-random,resnet,none,0,7,59.424322843551636,77.70540714263916,87.10270524024963,92.82972812652588,96.13513350486755,97.85135388374329
294,Exp-out1-random,resnet,none,0,8,59.18648838996887,77.5351345539093,86.92432641983032,92.62973070144653,96.0189163684845,97.65135049819946
295,Exp-out1-random,resnet,none,0,9,59.69189405441284,78.25405597686768,87.44324445724487,92.77567863464355,96.06486558914185,97.80270457267761


In [215]:
accuracies.loc[idx]['model']

318         LSTM
362          CNN
406    CNN_small
450     LSTM+CNN
494     CNN+LSTM
538          MLP
582       resnet
Name: model, dtype: object

In [252]:
model = 'MLP'
dataset = "Exp-out1-random"
preprocessing = 'CWT-H'

print(model)

idx = (accuracies.preprocessing==preprocessing) & (accuracies.kfold=='avg') & (accuracies.dataset == dataset)# & (accuracies.model == model
idx2 = (accuracies.preprocessing=='none') & (accuracies.kfold=='avg') & (accuracies.dataset == dataset)

lala = accuracies.loc[idx]
lala['model_cat'] = pd.Categorical(accuracies.loc[idx]['model'], categories = ['MLP', 'CNN_small', 'CNN', 'LSTM', 'CNN+LSTM', 'LSTM+CNN', 'resnet'], ordered=True)
lala = lala.sort_values('model_cat')

lolo = accuracies.loc[idx2]
lolo['model_cat'] = pd.Categorical(accuracies.loc[idx2]['model'], categories = ['MLP', 'CNN_small', 'CNN', 'LSTM', 'CNN+LSTM', 'LSTM+CNN', 'resnet'], ordered=True)
lolo = lolo.sort_values('model_cat')

(pd.to_numeric(lala.loc[:, 'accuracy'])).round(2)
#(pd.to_numeric(lala.loc[:, 'accuracy']).reset_index(drop=True) - pd.to_numeric(lolo.loc[:, 'accuracy']).reset_index(drop=True)).round(2)

MLP


/tmp/ipykernel_449109/1076979515.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lala['model_cat'] = pd.Categorical(accuracies.loc[idx]['model'], categories = ['MLP', 'CNN_small', 'CNN', 'LSTM', 'CNN+LSTM', 'LSTM+CNN', 'resnet'], ordered=True)
/tmp/ipykernel_449109/1076979515.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lolo['model_cat'] = pd.Categorical(accuracies.loc[idx2]['model'], categories = ['MLP', 'CNN_small', 'CNN', 'LSTM', 'CNN+LSTM', 'LSTM+CNN', 'resnet'], ordered=True)


560    54.63
428    53.12
384    54.15
340    50.68
516    45.36
472    55.01
604    54.94
Name: accuracy, dtype: float64

In [231]:
lala

,dataset,model,preprocessing,seed,kfold,accuracy,top-2-acc,top-3-acc,top-4-acc,top-5-acc,top-6-acc,model_cat
538,Exp-in-nops,MLP,CWT-H,0,avg,82.2475677728653,-,-,-,-,-,MLP
406,Exp-in-nops,CNN_small,CWT-H,0,avg,81.68594539165497,-,-,-,-,-,CNN_small
362,Exp-in-nops,CNN,CWT-H,0,avg,82.50810742378235,-,-,-,-,-,CNN
318,Exp-in-nops,LSTM,CWT-H,0,avg,77.26648569107056,-,-,-,-,-,LSTM
494,Exp-in-nops,CNN+LSTM,CWT-H,0,avg,68.1745958328247,-,-,-,-,-,CNN+LSTM
450,Exp-in-nops,LSTM+CNN,CWT-H,0,avg,83.48594605922699,-,-,-,-,-,LSTM+CNN
582,Exp-in-nops,resnet,CWT-H,0,avg,82.79216170310974,-,-,-,-,-,resnet


In [232]:
lolo

,dataset,model,preprocessing,seed,kfold,accuracy,top-2-acc,top-3-acc,top-4-acc,top-5-acc,top-6-acc,model_cat
230,Exp-in-nops,MLP,none,0,avg,79.24432396888733,92.5008112192154,97.13810801506042,98.96973013877869,99.66297268867493,99.83054161071777,MLP
98,Exp-in-nops,CNN_small,none,0,avg,82.38270282745361,94.09567594528198,97.8894591331482,99.31432485580444,99.81729686260223,99.92729783058167,CNN_small
54,Exp-in-nops,CNN,none,0,avg,84.9099999666214,95.41648745536804,98.50162267684937,99.55108165740967,99.89513576030731,99.95459496974945,CNN
10,Exp-in-nops,LSTM,none,0,avg,77.21216201782227,91.22351229190826,96.49648606777191,98.73486399650574,99.57405388355255,99.79378402233124,LSTM
186,Exp-in-nops,CNN+LSTM,none,0,avg,81.52864873409271,93.6908107995987,97.73891925811768,99.2527025938034,99.80783760547638,99.92108047008514,CNN+LSTM
142,Exp-in-nops,LSTM+CNN,none,0,avg,84.04432415962219,95.01053988933563,98.31621587276459,99.47513461112976,99.86054122447968,99.94675576686859,LSTM+CNN
274,Exp-in-nops,resnet,none,0,avg,86.45594716072083,96.07729732990265,98.74297320842743,99.64027047157288,99.91756796836853,99.96567606925964,resnet


In [222]:
accuracies.loc[idx]

,dataset,model,preprocessing,seed,kfold,accuracy,top-2-acc,top-3-acc,top-4-acc,top-5-acc,top-6-acc
318,Exp-in-nops,LSTM,CWT-H,0,avg,77.26648569107056,-,-,-,-,-
362,Exp-in-nops,CNN,CWT-H,0,avg,82.50810742378235,-,-,-,-,-
406,Exp-in-nops,CNN_small,CWT-H,0,avg,81.68594539165497,-,-,-,-,-
450,Exp-in-nops,LSTM+CNN,CWT-H,0,avg,83.48594605922699,-,-,-,-,-
494,Exp-in-nops,CNN+LSTM,CWT-H,0,avg,68.1745958328247,-,-,-,-,-
538,Exp-in-nops,MLP,CWT-H,0,avg,82.2475677728653,-,-,-,-,-
582,Exp-in-nops,resnet,CWT-H,0,avg,82.79216170310974,-,-,-,-,-


In [207]:
model = 'MLP'


print(model)

idx = (accuracies.preprocessing=='CWT-H') & (accuracies.kfold=='avg') & (accuracies.dataset == "Exp-in-nops")# & (accuracies.model == model)
pd.to_numeric(accuracies.loc[idx, 'accuracy']).round(2)

MLP


318    77.27
362    82.51
406    81.69
450    83.49
494    68.17
538    82.25
582    82.79
Name: accuracy, dtype: float64

In [220]:
accuracies_classical.loc[idx]

,dataset,model,preprocessing,accuracy


In [201]:
model = 'SVM'


print(model)

idx = (accuracies_classical.preprocessing=='LDA') & (accuracies_classical.model == model)
pd.to_numeric(accuracies_classical.loc[idx, 'accuracy']).round(2)

SVM


7     65.33
15    50.44
23    45.96
31    37.11
Name: accuracy, dtype: float64

In [130]:
print("JOBS TO REPEAT ("+str(len(jobs_to_repeat))+"):")
jobs_to_repeat.sort()
for job in jobs_to_repeat:
    print(job)

JOBS TO REPEAT (0):


In [9]:
sensor_tag="all"
seeds = [0]
preprocessings = ["none", "CWT-H", "CWT-V"]
models = ["LSTM", "CNN", "CNN_small", "LSTM+CNN", "CNN+LSTM", "MLP", "resnet"]
datasets = ["IN/nops", "OUT1/nops", "OUT1/random", "OUT2/nops"]
i = 10000

print("REMOVE JOBS THAT FAILED FROM NFS:")

for seed in seeds:
    for preprocessing in preprocessings:
        for model in models:
            for dataset in datasets:
                
                model_small=model.replace("_", "-").replace("+", "-").replace("/", "-").lower()
                dataset_small=dataset.replace("_", "-").replace("+", "-").replace("/", "-").lower()
                preprocessing_small=preprocessing.replace("_", "-").replace("+", "-").replace("/", "-").lower()

                out_path=acc_file="/tmp/drive/data/sakura/Exp-"+dataset+"/results/"+model+"_p"+preprocessing+"_"+str(i)+"_s"+str(seed)+"-sensors-"+sensor_tag+"_10fold"
                acc_file=out_path+"/out/kfold_results.txt"
                job_name = "prs-"+dataset_small+"-"+model_small+"-p"+preprocessing_small+"-"+str(i)+"-s"+str(seed)+"-sens-"+sensor_tag
            
                if job_name in jobs_to_repeat:
                    print("rm -r "+out_path)
                    

REMOVE JOBS THAT FAILED FROM NFS:


In [10]:
sensor_tag="all"
seeds = [0]
preprocessings = ["none", "CWT-H", "CWT-V"]
models = ["LSTM", "CNN", "CNN_small", "LSTM+CNN", "CNN+LSTM", "MLP", "resnet"]
datasets = ["IN/nops", "OUT1/nops", "OUT1/random", "OUT2/nops"]
i = 10000

print("REMOVE JOBS THAT FAILED LOCALLY:")

for seed in seeds:
    for preprocessing in preprocessings:
        for model in models:
            for dataset in datasets:
                
                model_small=model.replace("_", "-").replace("+", "-").replace("/", "-").lower()
                dataset_small=dataset.replace("_", "-").replace("+", "-").replace("/", "-").lower()
                preprocessing_small=preprocessing.replace("_", "-").replace("+", "-").replace("/", "-").lower()

                out_path=acc_file=OUT_DIR+"/sakura/Exp-"+dataset+"/results/"+model+"_p"+preprocessing+"_"+str(i)+"_s"+str(seed)+"-sensors-"+sensor_tag+"_10fold"
                acc_file=out_path+"/out/kfold_results.txt"
                job_name = "prs-"+dataset_small+"-"+model_small+"-p"+preprocessing_small+"-"+str(i)+"-s"+str(seed)+"-sens-"+sensor_tag
            
                if job_name in jobs_to_repeat:
                    print("rm -r "+out_path)

REMOVE JOBS THAT FAILED LOCALLY:


In [11]:
sensor_tag="all"
seeds = [0]
preprocessings = ["none", "CWT-H", "CWT-V"]
models = ["LSTM", "CNN", "CNN_small", "LSTM+CNN", "CNN+LSTM", "MLP", "resnet"]
datasets = ["IN/nops", "OUT1/nops", "OUT1/random", "OUT2/nops"]
i = 10000

print("JOB NAMES IN WANDB")

for seed in seeds:
    for preprocessing in preprocessings:
        for model in models:
            for dataset in datasets:
                
                model_weird = model.replace("+", "\+")
                model_small=model.replace("_", "-").replace("+", "-").replace("/", "-").lower()
                dataset_small=dataset.replace("_", "-").replace("+", "-").replace("/", "-").lower()
                preprocessing_small=preprocessing.replace("_", "-").replace("+", "-").replace("/", "-").lower()

                out_path=acc_file=OUT_DIR+"/sakura/Exp-"+dataset+"/results/"+model+"_p"+preprocessing+"_"+str(i)+"_s"+str(seed)+"-sensors-"+sensor_tag+"_10fold"
                acc_file=out_path+"/out/kfold_results.txt"
                job_name = "prs-"+dataset_small+"-"+model_small+"-p"+preprocessing_small+"-"+str(i)+"-s"+str(seed)+"-sens-"+sensor_tag
            
                if job_name in jobs_to_repeat:
                    print("Exp-"+dataset_small+"_"+model_weird+"_p"+preprocessing+"_"+str(i)+"_s"+str(seed)+"-sensors-"+sensor_tag)

JOB NAMES IN WANDB


In [ ]:
sensor_tag="all"
seeds = [0]
preprocessings = ["none", "CWT-H", "CWT-V"]
models = ["LSTM", "CNN", "CNN_small", "LSTM+CNN", "CNN+LSTM", "MLP", "resnet"]
datasets = ["IN/nops", "OUT1/nops", "OUT1/random", "OUT2/nops"]
i = 10000

print("REMOVE ALL UNZIPPED JOBS FROM S3")

for seed in seeds:
    for preprocessing in preprocessings:
        for model in models:
            for dataset in datasets:
                
                model_small=model.replace("_", "-").replace("+", "-").replace("/", "-").lower()
                dataset_small=dataset.replace("_", "-").replace("+", "-").replace("/", "-").lower()
                preprocessing_small=preprocessing.replace("_", "-").replace("+", "-").replace("/", "-").lower()

                out_path=acc_file=OUT_DIR+"/sakura/Exp-"+dataset+"/results/"+model+"_p"+preprocessing+"_"+str(i)+"_s"+str(seed)+"-sensors-"+sensor_tag+"_10fold"
                acc_file=out_path+"/out/kfold_results.txt"
                job_name = "prs-"+dataset_small+"-"+model_small+"-p"+preprocessing_small+"-"+str(i)+"-s"+str(seed)+"-sens-"+sensor_tag
                    
                print("rm -r "+out_path+"/out")

In [161]:
sensor_tag="all"
seeds = [0]
preprocessings = ["none", "CWT-H", "CWT-V"]
models = ["LSTM", "CNN", "CNN_small", "LSTM+CNN", "CNN+LSTM", "MLP", "resnet"]
datasets = ["IN/nops", "OUT1/nops", "OUT1/random", "OUT2/nops"]
i = 10000

columns = ['dataset', 'model', 'preprocessing', 'accuracy']
accuracies_classical = pd.DataFrame(columns=columns)

results_idx = 0

for dataset in datasets:

    dataset_small=dataset.replace("_", "-").replace("+", "-").replace("/", "-").lower()
    acc_file=OUT_DIR+"/sakura/Exp-"+dataset+"/results/classical/results_averaged.txt"

    # Check if job was completed
    if(os.path.exists(acc_file)!=True):
        print("Results not available: "+acc_file)
        continue

    file1 = open(acc_file, 'r')
    Lines = file1.readlines()

    for count, line in enumerate(Lines):
        #print("Line{}: {}".format(count, line.strip()))
        if((count % 2) == 1):
            model = str.split(line)[2]
            preprocessing = str.split(line)[3].split(';')[0]        
            kf_accuracy = float(re.findall("\d+\.\d+", line.strip())[0])*100
            accuracies_classical.loc[results_idx] = ["Exp-"+dataset_small, model, preprocessing, kf_accuracy]
            results_idx = results_idx+1

    file1.close()

In [163]:
accuracies_classical.to_csv(OUT_DIR+"/sakura/accuracies/accuracies_classical_averaged.csv", index=False)